In [18]:

import os
import math
import time
from datetime import datetime
import logging
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import yaml
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix

import datasource, causal_cnn_models, modules, net_utils

# Set-up

## Logger

In [2]:
logger = logging.getLogger(__name__)

def log(msg):
    logger.debug(msg)


def config_logger(log_file=None):
    r"""Config logger."""
    global logger
    logger.handlers.clear()
    logger.setLevel(logging.DEBUG)
    
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # create file handler which logs even debug messages
    if log_file:
        fh = logging.FileHandler(log_file)
        fh.setFormatter(format)
        logger.addHandler(fh)

def get_freer_gpu():
    # Run nvidia-smi command to get GPU memory info
    command = "nvidia-smi --query-gpu=memory.free,memory.total --format=csv,noheader,nounits"
    output = os.popen(command).read().strip().split('\n')

    # Parse the output to find GPU with most free memory
    max_memory = -1
    max_memory_index = -1
    for index, line in enumerate(output):
        free_memory, _ = map(int, line.split(', '))
        if free_memory > max_memory:
            max_memory = free_memory
            max_memory_index = index

    return max_memory_index

def viz_epoch_batch(epoch, i_batch, x_batch, x_hat_batch, log_path):
    # folder = os.path.join("logs", "recon_vae", log_filename)
    folder = os.path.join(log_path, "recon_vae")
    if not os.path.exists(folder):
        os.makedirs(folder)
    x_batch = x_batch.detach().cpu().numpy()
    x_hat_batch = x_hat_batch.detach().cpu().numpy()
    for i in range(1):
        orig = x_batch[i, 0, :]
        recon = x_hat_batch[i, 0, :]
        _, ax = plt.subplots()
        ax.plot(range(len(orig)), orig)
        # plt.savefig(
        #     f"{folder}/epoch{epoch}_item{i}_orig.png",
        #     format='png', dpi=300, bbox_inches='tight')
        ax.plot(range(len(recon)), recon)
        # plt.ylim((0, 2)) 
        plt.savefig(
            f"{folder}/epoch{epoch}_batch{i_batch}_item{i}.png",
            format='png', dpi=300, bbox_inches='tight')
    plt.close()

    
def save_models(model_file_instance_pairs):
    for model_file in model_file_instance_pairs.keys():
        net = model_file_instance_pairs.get(model_file)
        torch.save(net.state_dict(), model_file)

def load_models(model_file_instance_pairs, device="cpu"):
    for model_file in model_file_instance_pairs.keys():
        net = model_file_instance_pairs.get(model_file)
        net.load_state_dict(
            torch.load(model_file, map_location=device, weights_only=True))
    # No return seems necessary, in-memory models updated.

net_utils.fix_randomness()

In [3]:
params = None 
SIM_FILE = 'simSplitVaeParameterizer'
CFG_FILE = 'config.yml'
with open(CFG_FILE, 'r') as stream:
        params = yaml.safe_load(stream)
        params['seg_len'] = params['hz'] * params['seg_len_sec']
        params['decoder']['width'] = params['seg_len']

tm_sim_start = f"{datetime.now():%Y%m%d%H%M%S}"
params['tm_sim_start'] = tm_sim_start
log_path = f"logs/{SIM_FILE}_{CFG_FILE[:-4]}_{params['data_path'].replace('/','')}_split{params['n_split']}_ecg_{tm_sim_start}"
model_path = f"{log_path}/models"
log_file = (f"{log_path}/{tm_sim_start}.log")
if not os.path.exists(log_path):
        os.makedirs(log_path)
if not os.path.exists(model_path):
        os.makedirs(model_path)
logger = logging.getLogger(__name__)
config_logger(log_file)

# config_logger()

if torch.cuda.is_available():
        DEVICE = torch.device(f"cuda:{get_freer_gpu()}")
elif torch.mps.is_available():
        DEVICE = "mps"
else:
        DEVICE = "cpu"

log(params)

2024-09-26 11:24:23,934 - __main__ - DEBUG - {'batch_size': 32, 'cuda': 0, 'data_path': 'data/mesa/polysomnography/set2x10', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 10, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'n_class': 2, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'tm_sim_start': '20240926112423'}


## Data

In [8]:
"""Data source"""
BASEDATADIR = r'E://s222165064'

class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
# class_map = {0:0, 1:1, 2:1, 3:2, 4:2, 5:3}
n_class = len(set(class_map.values()))
params['n_class'] = n_class
log(f"class-map: {class_map}")
ds = datasource.MesaDb(
    f"{BASEDATADIR}/data/mesa/polysomnography", data_subdir="set1x20",
    hz=100, n_subjects=-1, hz_rr=params['hz_rr'], class_map=class_map, 
    is_rr_sig=False, is_rsp=False, is_ecg_beats=True, log=log,
)

2024-09-26 11:29:54,001 - __main__ - DEBUG - class-map: {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 2}
2024-09-26 11:29:54,003 - __main__ - DEBUG - Data base-dir:E://s222165064/data/mesa/polysomnography, data:set1x20, annot:annotations-events-nsrr, hz:100, seg_sec:30, class_map:{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 2}, n_classes:3
2024-09-26 11:29:54,005 - __main__ - DEBUG - Loading mesa-sleep-0046...


Extracting EDF parameters from E:\s222165064\data\mesa\polysomnography\set1x20\mesa-sleep-0046.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


2024-09-26 11:29:54,025 - __main__ - DEBUG - channels: ['EKG', 'EOG-L', 'EOG-R', 'EMG', 'EEG1', 'EEG2', 'EEG3', 'Pres', 'Flow', 'Snore', 'Thor', 'Abdo', 'Leg', 'Therm', 'Pos', 'EKG_Off', 'EOG-L_Off', 'EOG-R_Off', 'EMG_Off', 'EEG1_Off', 'EEG2_Off', 'EEG3_Off', 'Pleth', 'OxStatus', 'SpO2', 'HR', 'DHR']


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2024-09-26 11:29:54,893 - __main__ - DEBUG - channels: ['EKG'], search:ekg, src_hz:256
2024-09-26 11:30:01,584 - __main__ - DEBUG - [mesa-sleep-0046] 208 events, age:1
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarn

...

c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
2024-09-26 12:27:23,587 - __main__ - DEBUG - No label for annot '9' in mesa-sleep-1901.edf
2024-09-26 12:27:23,590 - __main__ - DEBUG - 	n_seg:1436, n_evt:214, annot_dist:{0: 665, 1: 178, 2: 483, 3: 9, 5: 104, 9: 1}, clz_lbl_dist:{0: 662, 1: 670, 2: 104}, remain:3000
2024-09-26 12:27:23,593 - __main__ - DEBUG - Loading mesa-sleep-1917...


Traceback (most recent call last):
  File "h:\Projects\split_vae\datasource.py", line 428, in _initialise
    seg = self.process_validate_segment(seg, f)
  File "h:\Projects\split_vae\datasource.py", line 485, in process_validate_segment
    'beats': get_beats(ecg_cleaned, self.hz, self.rr_seg_dim, n_beat=2) if self.is_ecg_beats else [],
  File "h:\Projects\split_vae\datasource.py", line 82, in get_beats
    beats = nk.ecg_segment(y, rpeaks=None, sampling_rate=hz, show=False)
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\ecg\ecg_segment.py", line 64, in ecg_segment
    heartbeats = epochs_create(
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\epochs\epochs_create.py", line 164, in epochs_create
    epoch_max_duration = int(max((i * sampling_rate for i in parameters["duration"])))
ValueError: cannot convert float NaN to integer

Extracting EDF parameters from E:\s222165064\data\mesa\polysomnography\set1x20\mesa-sleep-1917.ed

2024-09-26 12:27:23,671 - __main__ - DEBUG - channels: ['EKG', 'EOG-L', 'EOG-R', 'EMG', 'EEG1', 'EEG2', 'EEG3', 'Pres', 'Flow', 'Snore', 'Thor', 'Abdo', 'Leg', 'Therm', 'Pos', 'EKG_Off', 'EOG-L_Off', 'EOG-R_Off', 'EMG_Off', 'EEG1_Off', 'EEG2_Off', 'EEG3_Off', 'Pleth', 'OxStatus', 'SpO2', 'HR', 'DHR']


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


2024-09-26 12:27:25,088 - __main__ - DEBUG - channels: ['EKG'], search:ekg, src_hz:256
2024-09-26 12:27:38,169 - __main__ - DEBUG - [mesa-sleep-1917] 171 events, age:0
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
2024-09-26 12:31:16,331 - __main__ - DEBUG - 	n_seg:1435, n_evt:171, annot_dist:{0: 665, 1: 57, 2: 571, 3: 23, 5: 123}, clz_lbl_dist:{0: 661, 1: 651, 2: 123}, remain:2800
2024-09-26 12:31:16,339 - __main__ - DEBUG - Total files:20, n_seg:21742, distribution:(array([0, 1, 2]), array([ 9140, 10588,  2014], dtype=int64))


In [9]:
ds.record_names.sort()
log(ds.record_names)

n_rec = len(ds.record_names)
train_frac = math.ceil(n_rec*0.8)
test_frac = n_rec - train_frac
validation_frac = math.ceil(train_frac*0.1)
train_frac = train_frac - validation_frac
print(n_rec, train_frac, validation_frac, test_frac)

train_rec_names = ds.record_names[:train_frac]
validation_rec_names = ds.record_names[train_frac:train_frac+validation_frac]
test_rec_names = ds.record_names[train_frac+validation_frac:]
log(f"N ({n_rec}) train/val/test: {train_frac}/{validation_frac}/{test_frac}")
log(f"Train: {train_rec_names}, val: {validation_rec_names}, test:{test_rec_names}")

train_idx = []
for rec in train_rec_names:
    train_idx.extend(ds.record_wise_segments[rec])
validation_idx = []
for rec in validation_rec_names:
    validation_idx.extend(ds.record_wise_segments[rec])
test_idx = []
for rec in test_rec_names:
    test_idx.extend(ds.record_wise_segments[rec])

r"Data loaders"
train_dataset = datasource.PartialDataset(ds, seg_index=train_idx, shuffle=True)
data_loader_train = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)

val_dataset = datasource.PartialDataset(ds, seg_index=validation_idx, shuffle=True)
data_loader_val = torch.utils.data.DataLoader(
    dataset=val_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)

test_dataset = datasource.PartialDataset(ds, seg_index=test_idx, shuffle=False)
data_loader_test = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=params['batch_size'], shuffle=False, drop_last=True)
log(f"Data-loader size train: {len(data_loader_train)}, val: {len(data_loader_val)}, test: {len(data_loader_test)}")

2024-09-26 12:36:50,396 - __main__ - DEBUG - ['mesa-sleep-0046', 'mesa-sleep-0155', 'mesa-sleep-0236', 'mesa-sleep-0271', 'mesa-sleep-0312', 'mesa-sleep-0332', 'mesa-sleep-0419', 'mesa-sleep-0537', 'mesa-sleep-0558', 'mesa-sleep-0643', 'mesa-sleep-0744', 'mesa-sleep-0934', 'mesa-sleep-1010', 'mesa-sleep-1044', 'mesa-sleep-1604', 'mesa-sleep-1789', 'mesa-sleep-1790', 'mesa-sleep-1803', 'mesa-sleep-1901', 'mesa-sleep-1917']
2024-09-26 12:36:50,399 - __main__ - DEBUG - N (20) train/val/test: 14/2/4
2024-09-26 12:36:50,402 - __main__ - DEBUG - Train: ['mesa-sleep-0046', 'mesa-sleep-0155', 'mesa-sleep-0236', 'mesa-sleep-0271', 'mesa-sleep-0312', 'mesa-sleep-0332', 'mesa-sleep-0419', 'mesa-sleep-0537', 'mesa-sleep-0558', 'mesa-sleep-0643', 'mesa-sleep-0744', 'mesa-sleep-0934', 'mesa-sleep-1010', 'mesa-sleep-1044'], val: ['mesa-sleep-1604', 'mesa-sleep-1789'], test:['mesa-sleep-1790', 'mesa-sleep-1803', 'mesa-sleep-1901', 'mesa-sleep-1917']
2024-09-26 12:36:50,421 - __main__ - DEBUG - Data-lo

20 14 2 4
label distribution: ['0:5778', '1:7208', '2:1366']
label distribution: ['0:1216', '1:1188', '2:287']
label distribution: ['0:2146', '1:2192', '2:361']


## Model

In [11]:
"""prepare model"""
params['n_class'] = n_class
params_decoder = params['decoder'].copy()
params_decoder['width'] = params['hz'] * params['seg_len_sec']

net = causal_cnn_models.FoldVaeClassifFoldWeightParameterizer(
    params['encoder'], params_decoder, n_split=params['n_split'], 
    n_class=params['n_class'], log=log, debug=True,
)

log(net)
log(f"# params total: {net_utils.count_parameters(net)}")

x = torch.randn(32, 1, params['seg_len'])

outputs = net(x)    
recon_x = outputs['x_hat']
clz_proba = outputs['clz_proba']

print(f"recon: {recon_x.shape}, proba:{clz_proba.shape}")


2024-09-26 12:37:16,128 - __main__ - DEBUG - FoldVaeClassifFoldWeightParameterizer(
  (encoder): CausalCNNVEncoder(
    (network): Sequential(
      (0): CausalCNN(
        (network): Sequential(
          (0): CausalConvolutionBlock(
            (causal): Sequential(
              (0): Conv1d(1, 128, kernel_size=(5,), stride=(1,), padding=(4,))
              (1): Chomp1d()
              (2): LeakyReLU(negative_slope=0.01)
              (3): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(4,))
              (4): Chomp1d()
              (5): LeakyReLU(negative_slope=0.01)
            )
            (upordownsample): Conv1d(1, 128, kernel_size=(1,), stride=(1,))
          )
          (1): CausalConvolutionBlock(
            (causal): Sequential(
              (0): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(8,), dilation=(2,))
              (1): Chomp1d()
              (2): LeakyReLU(negative_slope=0.01)
              (3): Conv1d(128, 128, kernel_size=(5,), stride=(1

--CausalCNNVDecoder linear1:torch.Size([32, 64])
--CausalCNNVDecoder linear2:torch.Size([32, 19200])
--CausalCNNVDecoder reshape:torch.Size([32, 64, 300])


2024-09-26 12:37:19,011 - __main__ - DEBUG - DEC x_split[0]:torch.Size([32, 1, 300]), _x_hat:torch.Size([32, 1, 300]), x_hat:torch.Size([32, 1, 300])


--CausalCNNVDecoder out_causal_cnn:torch.Size([32, 1, 300])


2024-09-26 12:37:20,181 - __main__ - DEBUG - --VAE enc_mu:torch.Size([32, 32]), enc_sd:torch.Size([32, 32])
2024-09-26 12:37:20,187 - __main__ - DEBUG - --VAE _z:torch.Size([32, 32]), enc_mu:torch.Size([32, 2, 32]), enc_sd:torch.Size([32, 2, 32])
2024-09-26 12:37:20,189 - __main__ - DEBUG - ENC x_split[1]:torch.Size([32, 1, 300]), _z:torch.Size([32, 32]), z:torch.Size([32, 64])
2024-09-26 12:37:20,563 - __main__ - DEBUG - DEC x_split[1]:torch.Size([32, 1, 300]), _x_hat:torch.Size([32, 1, 300]), x_hat:torch.Size([32, 1, 600])
2024-09-26 12:37:20,785 - __main__ - DEBUG - --VAE enc_mu:torch.Size([32, 32]), enc_sd:torch.Size([32, 32])
2024-09-26 12:37:20,789 - __main__ - DEBUG - --VAE _z:torch.Size([32, 32]), enc_mu:torch.Size([32, 3, 32]), enc_sd:torch.Size([32, 3, 32])
2024-09-26 12:37:20,792 - __main__ - DEBUG - ENC x_split[2]:torch.Size([32, 1, 300]), _z:torch.Size([32, 32]), z:torch.Size([32, 96])
2024-09-26 12:37:22,408 - __main__ - DEBUG - DEC x_split[2]:torch.Size([32, 1, 300]), _x

recon: torch.Size([32, 1, 3000]), proba:torch.Size([32, 3])


# Model Training

In [14]:
def calculate_recon_loss(
        criteria_recon, recon_net, input, x_hat):
    loss_recon = 1. * criteria_recon(x_hat.squeeze(1), input.squeeze(1))
    elbo = 1. * net.kl
    loss_recon =  loss_recon + elbo    
    return loss_recon


def calculate_classif_loss(criteria_classif, cls_proba, labels):
    loss_classif = 1*criteria_classif(cls_proba, labels)
    return loss_classif

writer = SummaryWriter()

r"Prepare model training"
model_files = [
    f"{model_path}/fold0_net.pt", 
]
model_instances = [
    net, 
]

class_weights = torch.from_numpy(net_utils.get_class_weights(
    [ds.seg_labels[i] for i in train_idx], n_class=n_class, log=log
)[-1]).type(torch.FloatTensor).to(DEVICE)

optimizer = torch.optim.Adam(net.parameters(), lr=params['lr'])

criteria_classif = nn.CrossEntropyLoss(weight=class_weights)
criteria_recon = nn.BCELoss()

r"Model training"
alpha = 500.
lambda_1 = 200.
lambda_2 = 0.
min_val_loss = 1000.
net.to(DEVICE)

for epoch in range(params['max_epoch']):
    since = time.time()
    epoch_recon_loss, epoch_classif_loss = 0., 0.
    
    net.train()
    # net_classif.train()
    for i_batch, batch_data in enumerate(data_loader_train):

        inputs = batch_data['ecg'].to(DEVICE)
        labels = batch_data['label'].to(DEVICE)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        x_hats = outputs['x_hat']
        z = outputs['z']
        cls_proba = outputs['clz_proba'] 

        if epoch % 5 == 0 and i_batch == 0:
            viz_epoch_batch(epoch, i_batch, inputs, x_hats, log_path)

        loss_recon = calculate_recon_loss(criteria_recon, net, inputs, x_hats)
        loss_classif = calculate_classif_loss(criteria_classif, cls_proba, labels)
        
        total_loss = alpha*loss_recon + lambda_1*loss_classif
        total_loss.backward()
        optimizer.step()

        epoch_recon_loss += loss_recon.detach().cpu().numpy()
        epoch_classif_loss += loss_classif.detach().cpu().numpy()

    
    time_elapsed = time.time() - since
    epoch_recon_loss = epoch_recon_loss / len(data_loader_train)
    epoch_classif_loss = epoch_classif_loss / len(data_loader_train)
    writer.add_scalar("Loss/Reconstruction/train", epoch_recon_loss, epoch)
    writer.add_scalar("Loss/Classification/train", epoch_classif_loss, epoch)

    val_loss = 0.
    
    net.eval()
    # net_classif.eval()
    with torch.no_grad():
        for batch_data in data_loader_val:
            inputs = batch_data['ecg'].to(DEVICE)
            labels = batch_data['label'].to(DEVICE)

            outputs = net(inputs)
            x_hats = outputs['x_hat']
            z = outputs['z']
            cls_proba = outputs['clz_proba'] 

            loss = calculate_classif_loss(criteria_classif, cls_proba, labels)
            val_loss += loss.detach().cpu().numpy()
        val_loss = val_loss / len(data_loader_val)

    if val_loss < min_val_loss:
        save_models({
            model_files[0]: model_instances[0],
        })
        # torch.save(net.state_dict(), model_file)
        log(f"Val loss updated {min_val_loss} -> {val_loss}")
        min_val_loss = val_loss
    writer.add_scalar("Loss/Total/validation", val_loss, epoch)
    log(
        f"epoch:{epoch}, loss-recon:{epoch_recon_loss:.5f}, loss-classif:{epoch_classif_loss:.5f}, "
        f"time:{time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s"
    )

2024-09-26 12:44:05,675 - __main__ - DEBUG - freq:[5778. 7208. 1366.], weights:[0.23641398 0.18951165 1.        ]
2024-09-26 12:49:01,311 - __main__ - DEBUG - Val loss updated 1000.0 -> 1.1309087240979785
2024-09-26 12:49:01,314 - __main__ - DEBUG - epoch:0, loss-recon:0.70347, loss-classif:0.97304, time:4m 39s
2024-09-26 12:54:01,195 - __main__ - DEBUG - epoch:1, loss-recon:0.68746, loss-classif:0.94578, time:4m 42s
2024-09-26 12:59:01,440 - __main__ - DEBUG - epoch:2, loss-recon:0.68517, loss-classif:0.93166, time:4m 43s
2024-09-26 13:04:00,603 - __main__ - DEBUG - epoch:3, loss-recon:0.68425, loss-classif:0.92471, time:4m 42s
2024-09-26 13:09:00,227 - __main__ - DEBUG - Val loss updated 1.1309087240979785 -> 1.056599693638938
2024-09-26 13:09:00,232 - __main__ - DEBUG - epoch:4, loss-recon:0.68437, loss-classif:0.92108, time:4m 42s
2024-09-26 13:14:01,183 - __main__ - DEBUG - epoch:5, loss-recon:0.68475, loss-classif:0.91006, time:4m 44s
2024-09-26 13:19:01,351 - __main__ - DEBUG - 

# Model Evaluation

In [15]:
r"Validation"
def init_load_models(params):
    """prepare model"""
    params_decoder = params['decoder'].copy()
    params_decoder['width'] = params['hz'] * params['seg_len_sec']
    net = causal_cnn_models.FoldVaeClassifFoldWeight(
        params['encoder'], params_decoder, n_split=params['n_split'], 
        n_class=params['n_class'], log=log, debug=False,
    )

    model_files = [
        f"{params['model_path']}/fold0_net.pt", 
    ]
    model_instances = [net]
    load_models({
        model_files[0]: model_instances[0].to(DEVICE),
    }, device=DEVICE)
    net.eval()
    return model_instances

In [16]:
r"Validation"
def init_load_models_z(params):
    """prepare model"""
    params_decoder = params['decoder'].copy()
    params_decoder['width'] = params['hz'] * params['seg_len_sec']
    net = causal_cnn_models.FoldVaeClassifFoldWeightParameterizer(
        params['encoder'], params_decoder, n_split=params['n_split'], 
        n_class=params['n_class'], log=log, debug=False,
    )

    model_files = [
        f"{params['model_path']}/fold0_net.pt", 
    ]
    model_instances = [net]
    load_models({
        model_files[0]: model_instances[0].to(DEVICE),
    }, device=DEVICE)
    net.eval()
    return model_instances


In [17]:
test_rec_names = ['mesa-sleep-1790', 'mesa-sleep-1803', 'mesa-sleep-1901', 'mesa-sleep-1917']
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
ds = datasource.MesaDb(
    f"{BASEDATADIR}/data/mesa/polysomnography", data_subdir="set1x20",
    hz=100, class_map=class_map, n_subjects=-1, hz_rr=5, 
    filter_records=test_rec_names,
    is_rr_sig=False, is_rsp=False, is_ecg_beats=True,
)

Data base-dir:E://s222165064/data/mesa/polysomnography, data:set1x20, annot:annotations-events-nsrr, hz:100, seg_sec:30, class_map:{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 2}, n_classes:3
Loading mesa-sleep-1790...
Extracting EDF parameters from E:\s222165064\data\mesa\polysomnography\set1x20\mesa-sleep-1790.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
channels: ['EKG', 'EOG-L', 'EOG-R', 'EMG', 'EEG1', 'EEG2', 'EEG3', 'Pres', 'Flow', 'Snore', 'Thor', 'Abdo', 'Leg', 'Therm', 'Pos', 'EKG_Off', 'EOG-L_Off', 'EOG-R_Off', 'EMG_Off', 'EEG1_Off', 'EEG2_Off', 'EEG3_Off', 'Pleth', 'OxStatus', 'SpO2', 'HR', 'DHR']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
channels: ['EKG'], search:ekg, src_hz:256
[mesa-sleep-1790] 334 events, age:0


c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


Traceback (most recent call last):
  File "h:\Projects\split_vae\datasource.py", line 428, in _initialise
    seg = self.process_validate_segment(seg, f)
  File "h:\Projects\split_vae\datasource.py", line 485, in process_validate_segment
    'beats': get_beats(ecg_cleaned, self.hz, self.rr_seg_dim, n_beat=2) if self.is_ecg_beats else [],
  File "h:\Projects\split_vae\datasource.py", line 82, in get_beats
    beats = nk.ecg_segment(y, rpeaks=None, sampling_rate=hz, show=False)
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\ecg\ecg_segment.py", line 64, in ecg_segment
    heartbeats = epochs_create(
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\epochs\epochs_create.py", line 164, in epochs_create
    epoch_max_duration = int(max((i * sampling_rate for i in parameters["duration"])))
ValueError: cannot convert float NaN to integer



c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


Traceback (most recent call last):
  File "h:\Projects\split_vae\datasource.py", line 428, in _initialise
    seg = self.process_validate_segment(seg, f)
  File "h:\Projects\split_vae\datasource.py", line 485, in process_validate_segment
    'beats': get_beats(ecg_cleaned, self.hz, self.rr_seg_dim, n_beat=2) if self.is_ecg_beats else [],
  File "h:\Projects\split_vae\datasource.py", line 82, in get_beats
    beats = nk.ecg_segment(y, rpeaks=None, sampling_rate=hz, show=False)
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\ecg\ecg_segment.py", line 64, in ecg_segment
    heartbeats = epochs_create(
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\epochs\epochs_create.py", line 164, in epochs_create
    epoch_max_duration = int(max((i * sampling_rate for i in parameters["duration"])))
ValueError: cannot convert float NaN to integer

	n_seg:1196, n_evt:334, annot_dist:{0: 544, 2: 358, 1: 145, 3: 66, 5: 86}, clz_lbl_dist:{0: 541, 

...

Traceback (most recent call last):
  File "h:\Projects\split_vae\datasource.py", line 428, in _initialise
    seg = self.process_validate_segment(seg, f)
  File "h:\Projects\split_vae\datasource.py", line 485, in process_validate_segment
    'beats': get_beats(ecg_cleaned, self.hz, self.rr_seg_dim, n_beat=2) if self.is_ecg_beats else [],
  File "h:\Projects\split_vae\datasource.py", line 82, in get_beats
    beats = nk.ecg_segment(y, rpeaks=None, sampling_rate=hz, show=False)
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\ecg\ecg_segment.py", line 64, in ecg_segment
    heartbeats = epochs_create(
  File "c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\epochs\epochs_create.py", line 164, in epochs_create
    epoch_max_duration = int(max((i * sampling_rate for i in parameters["duration"])))
ValueError: cannot convert float NaN to integer

Traceback (most recent call last):
  File "h:\Projects\split_vae\datasource.py", line 428, in _in

c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


channels: ['EKG'], search:ekg, src_hz:256
[mesa-sleep-1917] 171 events, age:0


c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\s222165064\.conda\envs\split_vae\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2


	n_seg:1435, n_evt:171, annot_dist:{0: 665, 1: 57, 2: 571, 3: 23, 5: 123}, clz_lbl_dist:{0: 661, 1: 651, 2: 123}, remain:2800
Total files:4, n_seg:4699, distribution:(array([0, 1, 2]), array([2146, 2192,  361], dtype=int64))


In [19]:

params = None 
CFG_FILE = 'config.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 10
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simSplitVaeParameterizer_config_datamesapolysomnographyset2x10_split10_ecg_20240926112423/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models_z(params)

for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        # if i_batch > 20:
        #     break
    
    df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    PRED_PATH = params['model_path'].replace('models', 'preds')
    if not os.path.exists(PRED_PATH):
            os.makedirs(PRED_PATH)
    df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    cm = confusion_matrix(df['label'], df['pred'])
    print(cm)

2024-09-26 13:55:02,033 - __main__ - DEBUG - {'batch_size': 32, 'cuda': 0, 'data_path': 'data/mesa/polysomnography/set2x10', 'early_stop_delta': 0.001, 'early_stop_patience': 30, 'hz': 100, 'hz_rr': 5, 'lr': 0.001, 'lr_scheduler_patience': 10, 'max_epoch': 10, 'min_lr': '1e-6', 'seg_len_sec': 30, 'seg_len': 3000, 'n_split': 10, 'n_class': 3, 'encoder': {'in_channels': 1, 'channels': 128, 'depth': 5, 'reduced_size': 64, 'out_channels': 32, 'kernel_size': 5, 'dropout': 0.3, 'softplus_eps': 0.0001, 'sd_output': True}, 'decoder': {'k': 32, 'width': 3000, 'in_channels': 64, 'channels': 128, 'depth': 5, 'out_channels': 1, 'kernel_size': 5, 'gaussian_out': False, 'softplus_eps': 0.0001, 'dropout': 0.0}, 'model_path': 'logs/simSplitVaeParameterizer_config_datamesapolysomnographyset2x10_split10_ecg_20240926112423/models'}


label distribution: ['0:541', '1:569', '2:86']
1196 1196
--CausalCNNVDecoder linear1:torch.Size([1, 64])
--CausalCNNVDecoder linear2:torch.Size([1, 19200])
--CausalCNNVDecoder reshape:torch.Size([1, 64, 300])
--CausalCNNVDecoder out_causal_cnn:torch.Size([1, 1, 300])
[[514   6  21]
 [419  47 103]
 [ 66   4  16]]
label distribution: ['0:282', '1:302', '2:48']
632 632
[[183   0  99]
 [219   3  80]
 [ 36   0  12]]
label distribution: ['0:662', '1:670', '2:104']
1436 1436
[[419   2 241]
 [105   0 565]
 [  9   0  95]]
label distribution: ['0:661', '1:651', '2:123']
1435 1435
[[484  80  97]
 [ 63 161 427]
 [  7  22  94]]


In [20]:
results_path = 'logs/simSplitVaeParameterizer_config_datamesapolysomnographyset2x10_split10_ecg_20240926112423/preds'

# Initialize lists to store accuracy per subject
accuracies = []

# Iterate over all CSV files in the folder
for filename in os.listdir(results_path):
    if filename.endswith('.csv'):
        # Read each CSV file
        file_path = os.path.join(results_path, filename)
        df = pd.read_csv(file_path)
        
        # Ensure the CSV has the correct columns
        if 'label' in df.columns and 'pred' in df.columns:
            # Calculate accuracy for the current subject
            accuracy = (df['label'] == df['pred']).mean()
            print(f"Accuracy for {filename}: {accuracy:.2%}")
            
            # Store the accuracy
            accuracies.append(accuracy)
        else:
            print(f"Skipping {filename}: required columns not found.")

# Calculate and print the average accuracy across all subjects
if accuracies:
    avg_accuracy = sum(accuracies) / len(accuracies)
    print(f"\nAverage accuracy across all subjects: {avg_accuracy:.2%}")
else:
    print("No valid CSV files found.")

Accuracy for mesa-sleep-1790.csv: 48.24%
Accuracy for mesa-sleep-1803.csv: 31.33%
Accuracy for mesa-sleep-1901.csv: 35.79%
Accuracy for mesa-sleep-1917.csv: 51.50%

Average accuracy across all subjects: 41.72%


In [ ]:

params = None 
CFG_FILE = 'config_multimodal_ecg.yml'
with open(CFG_FILE, 'r') as stream:
    params = yaml.safe_load(stream)
    params['seg_len'] = params['hz'] * params['seg_len_sec']
    params['decoder']['width'] = params['seg_len']
    # override specific params
    params['n_split'] = 5
    params['encoder']['depth'] = 5
    params['decoder']['depth'] = 5
    
config_logger()
params['model_path'] = "logs/simMultimodalEcgNoAge_config_multimodal_ecg_datamesapolysomnographyset1_split5_ecgTrue_rrFalse_rspFalse_20240922095352/models"
class_map = {0:0, 1:1, 2:1, 3:1, 4:1, 5:2}
params['n_class'] = len(set(class_map.values()))
log(params)

DEVICE = torch.device(f"cuda:{params['cuda']}" if torch.cuda.is_available() else "cpu")

networks = init_load_models(params)
# print(networks)


for test_rec in test_rec_names:
    p_ds = datasource.PartialDataset(
        dataset=ds, seg_index=ds.record_wise_segments[test_rec]
    )
    data_loader_test = torch.utils.data.DataLoader(
            dataset=p_ds, batch_size=1, shuffle=False, drop_last=False)
    print(len(p_ds), len(data_loader_test))

    outputs = []
    tp, tn = 0, 0
    for i_batch, batch_data in enumerate(data_loader_test):
        input_ecg = batch_data['ecg'].to(DEVICE)
        label = batch_data['label'].detach().cpu().numpy()[0]
        net_outputs = networks[0](input_ecg)
        cls_proba = net_outputs['clz_proba'] 
        w_folds = net_outputs['w_folds']
        # cls_proba = networks[1](z)
        pred_label = torch.argmax(cls_proba, axis=1).detach().cpu().numpy()[0]
        # print(label, pred_label)
        outputs.append({
            'label': label, 'pred': pred_label
        })
        viz(batch_data, net_outputs, pred_label)
        
        if i_batch > 0:
            break
    
    # df = pd.DataFrame.from_dict(outputs)
    # print(df)
    # break

    # PRED_PATH = params['model_path'].replace('models', 'preds')
    # if not os.path.exists(PRED_PATH):
    #         os.makedirs(PRED_PATH)
    # df.to_csv(f"{PRED_PATH}/{test_rec}.csv")

    # cm = confusion_matrix(df['label'], df['pred'])
    # print(cm)